In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaModel
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import time
import datetime
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import math

In [ ]:
'''
Dataset Class, returns standard inputs for transformer (input_ids, attention_mask, label),
and  ancillary_input (secondary data if training transformer-CNN)
train_data: dataset (dataframe), labels: data labels
tokenizer: tokenizer, max_len: max sequence length
'''
class TextDataset(Dataset):

    def __init__(self, train_data, labels, tokenizer, max_len):
        self.clinical_history = train_data['Clinical history'].values.tolist()
        self.ancillary_data = train_data.copy().drop(['Protocol', 'Clinical history'], axis=1).values.tolist()
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.clinical_history)

    def __getitem__(self, idx):
        text = self.clinical_history[idx]
        label = self.labels[idx]
        ancillary_data = self.ancillary_data[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_len, padding='max_length',
                                  truncation=True, add_special_tokens=True)

        return {'input_ids': encoding['input_ids'].unsqueeze(0).flatten(),
                'attention_mask': encoding['attention_mask'].unsqueeze(0).flatten(),
                'label': torch.tensor(label),
                'ancillary_input': torch.tensor(ancillary_data).unsqueeze(0).flatten()}


In [ ]:
'''
RoBERTa classifier (no CNN)
model: RoBERTa model name, classes: number of classes
'''
class RobertaClassifier(nn.Module):

    def __init__(self, model, classes):
        super(RobertaClassifier, self).__init__()
        self.roberta = RobertaForSequenceClassification.from_pretrained(model, num_labels=classes)

    def forward(self, input_ids, attention_mask, labels):
        return self.roberta(input_ids=input_ids, attention_mask=attention_mask, labels=labels)



In [ ]:
'''
Combined RoBERTa classifier and CNN
r_model: RoBERTa model name, classes: number of classes, feature_len: length of ancillary data
'''
class RobertaCNNClassifier(nn.Module):

    def __init__(self, r_model, classes, feature_len):
        super(RobertaCNNClassifier, self).__init__()
        self.comb_len = feature_len + 768
        self.roberta = RobertaModel.from_pretrained(r_model)
        self.pre_conv = nn.Linear(self.comb_len, self.comb_len)

        # Convolutional layers
        self.conv1d_1 = nn.Conv1d(1, 16, kernel_size=3)
        self.conv1d_2 = nn.Conv1d(16, 32, kernel_size=3)
        self.conv1d_3 = nn.Conv1d(32, 16, kernel_size=3)

        # Dropout, MaxPool and Fully connected layers
        self.dropout = nn.Dropout(0.5)
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.pre_classifier = nn.Linear(2256, 1024)
        self.classifier = nn.Linear(1024, classes)

    def forward(self, input_ids, attention_mask, labels, ancillary_data):
        output1 = self.roberta(input_ids=input_ids, attention_mask=attention_mask) # pass inputs to roberta model
        hidden_state = output1.last_hidden_state
        hidden_state = hidden_state[:, 0, :] # get CLS tokens from last hidden state
        x = torch.concat((hidden_state, ancillary_data), dim=1)  #concatenate transformer output and ancillary ohe data
        x = self.pre_conv(x)  # batch_size=32, inputs=comb_len
        x = x.unsqueeze(2)
        x = x.permute(0, 2, 1)  # batch_size=32, channels=1, length=comb_len

        x = self.conv1d_1(x)
        x = nn.ReLU()(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        x = self.conv1d_2(x)
        x = nn.ReLU()(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        x = self.conv1d_3(x)
        x = nn.ReLU()(x)
        x = self.maxpool(x)
        x = self.dropout(x)

        x = x.view(x.size(0), -1)  # re-shape to batch_size, channels x length
        x = self.pre_classifier(x)
        x = nn.ReLU()(x)
        x = self.dropout(x)
        output = self.classifier(x)
        return output


In [ ]:
'''
Focal loss function defined as -self.alpha * (1 - pt) ** self.gamma) * log_prob
Tunable hyperparameters alpha, gamma and weights as described in Methods section.
'''
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, weights=None):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.weights = weights

    def forward(self, inputs, targets):
        log_prob = F.log_softmax(inputs, dim=-1)
        pt = torch.exp(log_prob)

        if self.weights != None:
          loss = F.nll_loss(
              (-self.alpha * (1 - pt) ** self.gamma) * log_prob,
              targets,
              weight=self.weights.float(),
              reduction = 'none')
        else:
          loss = F.nll_loss(
              ((1 - pt) ** self.gamma) * log_prob,
              targets,
              reduction = 'none')
        return loss.mean()

In [ ]:
'''
Takes one hot encoded labels list and returns weights.
labels_df: dataframe of data labels
method: weighting method, can be IF, ISF or ENS.
beta: ENS hyperparameter
'''
def get_weights_loss_function(labels_df, method, beta=None):

    label_counts = labels_df.value_counts()
    label_counts = label_counts.sort_index()

    if method == "IF":
      weights = 1.0 / np.array(label_counts)
      weights = weights / np.sum(weights) * len(label_counts)
    elif method == "ISF":
      weights = 1.0 / np.array(np.power(label_counts, 0.5))
      weights = weights / np.sum(weights) * len(label_counts)
    elif method == "ENS":
      weights = (1.0 - beta) / np.array(1.0 - np.power(beta, label_counts))
      weights = weights / np.sum(weights) * len(label_counts)

    return np.float32(weights)

In [ ]:
'''
parameters
train_model: Training model
train_data_loader: DataLoader object for training dataset
criterion: Loss function
train_optimizer: Optimizer
scheduler: Learning rate scheduler
train_device: cuda
cnn: True if using transformer-CNN
'''
def train(train_model, train_dataloader, criterion, train_optimizer, scheduler, train_device, cnn=None):
    train_model.train()
    y_pred = []
    y_true = []
    train_loss = []

    # Reset total loss
    print("Training...")

    for batch in tqdm(train_dataloader):
        # Pass tensors to device, includes ancillary secondary data if using Roberta-CNN classifier
        input_ids = batch['input_ids'].to(train_device)
        attention_mask = batch['attention_mask'].to(train_device)
        labels = batch['label'].to(train_device)
        if cnn:
          ancillary_input = batch['ancillary_input'].to(train_device)

        # Clear previously calculated gradients
        optimizer.zero_grad()

        # Forward pass, includes ancillary secondary data if using Roberta-CNN classifier
        if cnn:
          outputs = train_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, ancillary_data=ancillary_input)
        else:
          outputs = train_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
          outputs = outputs.logits

        # Loss function - CrossEntropy, calculate total loss
        loss = criterion(outputs, labels)
        train_loss.append(loss.item())

        # Move prediction and label tensors to cpu, and convert to lists
        y_pred.extend(outputs.cpu().tolist())
        y_true.extend(labels.cpu().tolist())

        # Backward pass
        loss.backward()

        # Update parameters and increment stepwise
        train_optimizer.step()

        # Update learning rate
        scheduler.step()

    # Calculate average_loss
    average_loss = np.array(train_loss).mean()

    # Calculate accuracy and sklearn classification report
    # y_true = np.argmax(y_true, axis=1)
    y_pred = np.argmax(y_pred, axis=1)

    accuracy = accuracy_score(y_true, y_pred)
    classification = classification_report(y_true, y_pred)

    print(f"Classification:\n{classification}")
    print(f"Average training accuracy: {accuracy}")
    print(f"Average training loss: {average_loss:.2f}")

    return accuracy, classification, average_loss


In [ ]:
'''
parameters
eval_model: Evaluation model
val_data_loader: DataLoader object for validation dataset
criterion: Loss function
eval_device: cuda
cnn: True if using transformer-CNN
'''
def evaluate(eval_model, val_dataloader, criterion, eval_device, cnn=None):

    eval_model.eval()
    val_loss = []
    y_pred = []
    y_true = []
    print("Validating...")

    # Deactivate autograd engine to reduce memory usage
    # with torch.no_grad():
    with torch.inference_mode():

        for batch in tqdm(val_dataloader):
            # Pass tensors to device, includes ancillary secondary data if using Roberta-CNN classifier
            input_ids = batch['input_ids'].to(eval_device)
            attention_mask = batch['attention_mask'].to(eval_device)
            labels = batch['label'].to(eval_device)
            if cnn:
              ancillary_input = batch['ancillary_input'].to(eval_device)

            # Forward pass, includes ancillary secondary data if using Roberta-CNN classifier
            if cnn:
              outputs = eval_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, ancillary_data=ancillary_input)
            else:
              outputs = eval_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
              outputs = outputs.logits

            # Get loss from outputs
            # Loss function - CrossEntropy, calculate total loss
            loss = criterion(outputs, labels)
            val_loss.append(loss.item())

            # Move prediction and label tensors to cpu, and convert to lists
            y_pred.extend(outputs.cpu().tolist())
            y_true.extend(labels.cpu().tolist())

    # Calculate average loss
    average_loss = np.array(val_loss).mean()

    # Calculate accuracy and sklearn classification report
    # y_true = np.argmax(y_true, axis=1)
    y_pred = np.argmax(y_pred, axis=1)

    accuracy = accuracy_score(y_true, y_pred)
    classification = classification_report(y_true, y_pred)

    print(f"Classification:\n{classification}")
    print(f"Average validation accuracy: {accuracy}")
    print(f"Average validation loss: {average_loss:.2f}")

    return accuracy, classification, average_loss


In [ ]:
'''
parameters
predict_model: Prediction (test) model
test_data_loader: DataLoader object for testing
predict_device: cuda
cnn: True if using transformer-CNN
'''
def predict(predict_model, test_data_loader, predict_device, cnn=None):
    predict_model.eval()
    y_pred = []
    y_true = []

    # Deactivate autograd engine to reduce memory usage
    # with torch.no_grad():
    with torch.inference_mode():

        for batch in tqdm(test_data_loader):
            # Pass tensors to device, includes ancillary secondary data if using Roberta-CNN classifier
            input_ids = batch['input_ids'].to(predict_device)
            attention_mask = batch['attention_mask'].to(predict_device)
            labels = batch['label'].to(predict_device)
            if cnn:
              ancillary_input = batch['ancillary_input'].to(predict_device)

            # Forward pass, includes ancillary secondary data if using Roberta-CNN classifier
            if cnn:
              outputs = predict_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, ancillary_data=ancillary_input)
            else:
              outputs = predict_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
              outputs = outputs.logits

            y_pred.extend(outputs.cpu().tolist())
            y_true.extend(labels.cpu().tolist())

    # y_true = np.argmax(y_true, axis=1)
    y_pred = np.argmax(y_pred, axis=1)

    return y_pred, y_true


In [ ]:
'''
LOAD DATA AND SET PARAMETERS
Load training and combined validation-test datasets from excel
Note this was previously split to allow data augmentation of training data only
set_aug: True will load augmented dataset, False non-augmented
set_cnn: True will load transformer-CNN model, False will load transformer model
dir_path: path to directory containing data, please change to your own directory
'''
set_aug = True
set_cnn = False
dir_path = "/content/drive/MyDrive/Colab Notebooks/Final Notebooks/Submission/"

if set_aug:
    X_train_df = pd.read_csv(dir_path + "Data/Xtrain_aug_1000-400.csv")
    X_val_test_df = pd.read_csv(dir_path + "/Data/Xtest_aug_1000-400.csv")

    y_train_df = pd.read_csv(dir_path + "Data/ytrain_aug_1000-400.csv")['Protocol']
    y_val_test_df = pd.read_csv(dir_path + "/Data/ytest_aug_1000-400.csv")['Protocol']

else:
    X_train_df = pd.read_csv(dir_path + "Data/Xtrain_70-30_no-aug.csv")
    X_val_test_df = pd.read_csv(dir_path + "/Data/Xtest_70-30_no-aug.csv")

    y_train_df = pd.read_csv(dir_path + "Data/ytrain_70-30_no-aug.csv")['Protocol']
    y_val_test_df = pd.read_csv(dir_path + "Data/ytest_70-30_no-aug.csv")['Protocol']

X_train_df = X_train_df.drop(['ID', 'Urg', 'UrgBin'], axis=1)
X_val_test_df = X_val_test_df.drop(['ID', 'Urg', 'UrgBin'], axis=1)

In [ ]:
print(X_train_df.head().to_string())

In [ ]:
# Calculate class weights to address data imbalance.
# Options for method = ["IF", "ISF", "ENS"], beta is tuneable hyperparameter tested at [0.9, 0.99, 0.999]
class_weights = get_weights_loss_function(y_train_df, method="ENS", beta=0.9)

# Get dimensions of all combined features - Protocol and ancillary
feat_len = X_train_df.copy().drop(['Clinical history', 'Protocol'], axis=1).shape[1]

In [ ]:
# Train, validation, test split
X_val_df, X_test_df, y_val_df, y_test_df = train_test_split(X_val_test_df, y_val_test_df, test_size=0.33, random_state=42)

y_train = y_train_df.values.tolist()

y_val = y_val_df.values.tolist()

y_test = y_test_df.values.tolist()

len(X_train_df), len(X_val_df), len(X_test_df)

In [ ]:
'''
MODEL HYPERPARAMETERS
Set Transformer hyperparameters, see manuscript for optimal settings
max_sequence_length optimised at 256, note 512 not feasible for all models due to GPU constraints
'''
if not set_cnn:
    num_classes = max(y_train) + 1
    batch_size = 16
    epochs = 3
    learning_rate = 2e-5
    warmup = 0.3
    max_sequence_length = 256
    patience = 5
    min_delta = 0.01

else:
    num_classes = max(y_train) + 1
    batch_size = 32
    epochs = 20
    learning_rate = 5e-4
    warmup = 0.2  # for learning rate scheduler
    max_sequence_length = 256
    patience = 5
    min_delta = 0.01

In [ ]:
# Set up RoBERTa tokenizer
model_name = '/content/drive/MyDrive/Colab Notebooks/huggingface_models/RoBERTa-base-PM-M3-Voc-distill-align-hf'
tokenizer = RobertaTokenizer.from_pretrained(model_name)

# Initialise datasets and dataloaders
train_dataset = TextDataset(X_train_df, y_train, tokenizer, max_sequence_length)
val_dataset = TextDataset(X_val_df, y_val, tokenizer, max_sequence_length)
test_dataset = TextDataset(X_test_df, y_test, tokenizer, max_sequence_length)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Set device, model, criterion, optimizer, warmup, scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if set_cnn:
    # Load previously trained RoBERTa model
    model = RobertaCNNClassifier(model_name, num_classes, feat_len).to(device)
    model.load_state_dict(torch.load(dir_path + "Models/protocol_roberta.pth"),
                          strict=False)

    #Freeze RoBERTa layers
    for name, param in model.named_parameters():
        if name.startswith("roberta"):
            param.requires_grad = False
else:
    model = RobertaClassifier(model_name, num_classes).to(device)

class_weights = torch.tensor(class_weights).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=learning_rate)

steps = len(train_dataloader) * epochs
warmup_steps = math.floor(steps * warmup)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=steps)


In [ ]:
# Set seed value
seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)

In [ ]:
# Train and validate model
train_val_stats = []

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_accuracy, train_classification, average_train_loss = train(
        model, train_dataloader, criterion, optimizer, scheduler, device, cnn=set_cnn)
    val_accuracy, val_classification, average_val_loss = evaluate(
        model, val_dataloader, criterion, device, cnn=set_cnn)

    # Append training and validation stats to list of dictionaries
    train_val_stats.append({
        'Epoch':epoch+1,
        'Training_Loss': average_train_loss,
        'Training_Accuracy': train_accuracy,
        'Validation_Loss': average_val_loss,
        'Validation_Accuracy': val_accuracy
    })

In [ ]:
# Create pandas dataframe from training and validation stats
df_stats = pd.DataFrame(data=train_val_stats)

df_stats = df_stats.set_index('Epoch')
epochs = df_stats.shape[0]

df_stats

In [ ]:
# Plot accuracy curves
plt.plot(df_stats['Training_Accuracy'], label='Train')
plt.plot(df_stats['Validation_Accuracy'], label='Val')

# Label accuracy plot
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.xticks(range(1,epochs+1))
plt.show()

# Plot loss curves
plt.plot(df_stats['Training_Loss'], label='Train')
plt.plot(df_stats['Validation_Loss'], label='Val')

# Label loss plot
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks(range(1,epochs+1))
plt.show()

In [ ]:
# Test model and print classification report

test_pred, test_true = predict(model, test_dataloader, device, cnn=set_cnn)
classification = classification_report(test_true, test_pred)
print(f"Test Classification:\n{classification}")

In [ ]:
'''
Saves model to google drive, commented out for submission

drive_path = '/content/drive/MyDrive/Colab Notebooks/Dissertation/models'
dt = datetime.datetime.now().replace(second=0, microsecond=0)
model_save_path = drive_path + f"/final_protocol_roberta.pth"

print(f"Saving to {model_save_path}")
torch.save(model.state_dict(), f=model_save_path)
'''

In [ ]:
#from google.colab import runtime
#runtime.unassign()